In [7]:
# Set up API key and do the necessary imports
from agentjo import *
import os

# this is only if you use OpenAI as your LLM
# os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [8]:
def add_item_to_inventory(shared_variables, item: str) -> str:
    ''' Adds item to inventory, and returns outcome of action '''
    shared_variables['Inventory'].append(item)
    return f'{item} successfully added to Inventory'
    
def remove_item_from_inventory(shared_variables, item: str) -> str:
    ''' Removes item from inventory and returns outcome of action '''
    if item in shared_variables['Inventory']:
        shared_variables['Inventory'].remove(item)
        return f'{item} successfully removed from Inventory'
    else:
        return f'{item} not found in Inventory, unable to remove'
    
agent = Agent('Inventory Manager', 
              'Adds and removes items in Inventory. Only able to remove items if present in Inventory',
              shared_variables = {'Inventory': []},
              global_context = 'Inventory: <Inventory>', # Add in Global Context here with shared_variables Inventory
              llm = llm).assign_functions([add_item_to_inventory, remove_item_from_inventory])

output = agent.run('Add apples and oranges')

Observation: The inventory is currently empty, and the task is to add apples and oranges to it.
Thoughts: To complete the task, I need to add both apples and oranges to the inventory. Since the inventory is empty, I can start by adding apples first.
Subtask identified: Add apples to the inventory.
Calling function add_item_to_inventory with parameters {'item': 'apples'}
> {'output_1': 'apples successfully added to Inventory'}

Observation: Apples have been successfully added to the inventory. The task requires adding both apples and oranges to the inventory.
Thoughts: Since apples have already been added, the next step is to add oranges to the inventory to complete the task.
Subtask identified: Add oranges to the inventory.
Calling function add_item_to_inventory with parameters {'item': 'oranges'}
> {'output_1': 'oranges successfully added to Inventory'}

Observation: The items apples and oranges have been successfully added to the inventory as per the subtasks completed.
Thoughts: The

In [9]:
from agentjo.agent import Agent,BaseAgent
from agentjo.base import strict_json
import re
from termcolor import colored

class CustomizationWrapper(Agent):
    def __init__(self,agent: Agent):
        # Initialize the parent Agent
        super().__init__(**agent.__dict__)  # Inherit all of the attributes of the passed agent

    def wrap_function(self, func, before_hook: list = [], after_hook: list = []):
            """
            Wraps a base agent function with before and after hooks.
            
            Args:
                func: The base agent function to wrap
                before_hook: List of callable functions to execute before the base function
                after_hook: List of callable functions to execute after the base function
            """
            
            # if functions are not already Function objects, convert all hooks to Function objects. If they are BaseAgent, convert them to Function objects
            before_hook = [hook.to_function(self) if isinstance(hook, BaseAgent) else Function(external_fn=hook) for hook in before_hook]
            after_hook = [hook.to_function(self) if isinstance(hook, BaseAgent) else Function(external_fn=hook) for hook in after_hook]
                    
            self.assign_functions(before_hook + after_hook)
            

            def infer_function_parameters(function: Function):
                input_format = {}
                fn_description = function.fn_description
                matches = re.findall(r'<(.*?)>', fn_description)
                
                # do up an output format dictionary to use to get LLM to output exactly based on keys and types needed
                for match in matches:
                    if ':' in match:
                        first_part, second_part = match.split(':', 1)
                        input_format[first_part] = f'A suitable value, type: {second_part}'
                    else:
                        input_format[match] = 'A suitable value'
                        
                # if there is no input, then do not need LLM to extract out function's input
                if input_format == {}:
                    function_params = {}
                        
                else:
                    background_info = f"Assigned Task:```\n{self.task}\n```\nSubtasks Completed: ```{self.subtasks_completed}```"
                    # Add in memory to the Agent
                    rag_info = ''
                    for name in self.memory_bank.keys():
                        # Function RAG is done separately in self.query()
                        if name == 'Function': continue
                    rag_info += f'Knowledge Reference for {name}: ```{self.memory_bank[name].retrieve(self.task)}```\n'    

                    function_params = self.query(query = f'''{background_info}{rag_info}\n\n```\nEquipped Function Details: ```{str(function)}```\nOutput suitable values for Inputs to Equipped Function to fulfil Current Subtask\nInput fields are: {list(input_format.keys())}''',
                                output_format = input_format,
                                provide_function_list = False)
                return function_params
            
            try:    
                # Get the original base function
                original_func = getattr(self, func)
            except:
                raise Exception(f"Base function: {func} not found in agent {self.agent_name}")
            
            def wrapped_function(*args, **kwargs):
                # Special handling for run function to ensure task is assigned before hooks
                if func == 'run' and len(args) > 0:
                    self.assign_task(args[0], args[1] if len(args) > 1 else '')
                    
                # Execute before hooks
                for hook in before_hook:
                    self.use_function(hook.fn_name, infer_function_parameters(hook))
                    
                # Execute original function
                result = original_func(*args, **kwargs)
                
                # Execute after hooks
                for hook in after_hook:
                    self.use_function(hook.fn_name, infer_function_parameters(hook))
                    
                return result
            
            # Replace the original function with wrapped version
            setattr(self, func, wrapped_function)

In [11]:
def add_item_to_inventory(shared_variables, item: str) -> str:
    ''' Adds item to inventory, and returns outcome of action '''
    shared_variables['Inventory'].append(item)
    return f'{item} successfully added to Inventory'
    
def remove_item_from_inventory(shared_variables, item: str) -> str:
    ''' Removes item from inventory and returns outcome of action '''
    if item in shared_variables['Inventory']:
        shared_variables['Inventory'].remove(item)
        return f'{item} successfully removed from Inventory'
    else:
        return f'{item} not found in Inventory, unable to remove'
    
agent = Agent('Inventory Manager', 
              'Adds and removes items in Inventory. Only able to remove items if present in Inventory',
              shared_variables = {'Inventory': []},
              global_context = 'Inventory: <Inventory>', # Add in Global Context here with shared_variables Inventory
              llm = llm)

agent = CustomizationWrapper(agent).assign_functions([add_item_to_inventory, remove_item_from_inventory])

def plan(shared_variables) -> str:
    '''Produces a plan for the task'''
    agent = shared_variables['agent']
    res = strict_json(system_prompt = f'''Your overall task is {agent.overall_task} \n Your available Equipped Functions are: {agent.function_map} \n Create a reasonable plan to accomplish the task in the form of a list of functions you wish to execute by name. ''',
                        user_prompt = '',
                        output_format = {'Plan': 'List of function names to run in order, type: list'},
                        llm = llm)
    print(colored(res['Plan'],'red',attrs=['bold']))
    return res['Plan']

def reflect(shared_variables) -> str:
    '''Produces a thought about the task with a little flavor'''
    agent = shared_variables['agent']
    res = strict_json(system_prompt = f'''Reflect on the progress towards your main task {agent.overall_task} \n Subtasks Completed: ```{agent.subtasks_completed}```''',
                        user_prompt = '',
                        output_format = {'Reflection': 'Your reflections, type: str'},
                        llm = llm)
    print(colored(res['Reflection'],'red',attrs=['bold']))
    return res['Reflection']

agent.wrap_function(func='run',before_hook = [plan])
agent.wrap_function(func='get_next_subtask',after_hook = [reflect])

output = agent.run('Add apples and oranges')

Calling function plan with parameters {}
['use_llm', 'add_item_to_inventory', 'end_task']
> {'output_1': ['use_llm', 'add_item_to_inventory', 'end_task']}

Observation: The plan has been created, indicating that the task involves using the LLM, adding items to the inventory, and then ending the task. However, the inventory is currently empty, and no items have been added yet.
Thoughts: To complete the task, I need to add the specified items, apples and oranges, to the inventory. Since the inventory is empty, I should start by adding apples first, followed by oranges.
Calling function reflect with parameters {}
The progress towards the main task of adding apples and oranges is on track. The subtasks identified include using a language model to assist in the process, adding the items to the inventory, and then concluding the task. This structured approach ensures that each step is clearly defined and can be executed efficiently. However, it is important to ensure that the language model 